# ДЗ 2. Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

## Подготовка данных

In [9]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize
import pymorphy2

from gensim.models import LdaModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix

In [10]:
news = pd.read_csv("data/articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [11]:
users = pd.read_csv("data/users_articles.csv")
print(users.shape)
users.head(3)

(8000, 2)


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [12]:
target = pd.read_csv("data/users_churn.csv")
print(target.shape)
target.head(3)

(8000, 2)


,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [17]:
stopword_ru = stopwords.words('russian')

morph = pymorphy2.MorphAnalyzer()

with open('data/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

151

In [18]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [19]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

c:\users\vlad\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


Wall time: 19.2 s


In [20]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 1min 33s


In [22]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [23]:
# Запускаем обучение
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)

In [24]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(3, 0.2730403), (7, 0.2724014), (11, 0.31866744), (19, 0.11672639)]

In [25]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: компания год топливо поверхность который турецкий издание
topic_1: рынок рост млн год объём доллар составить
topic_2: рубль год проект санкция закон правительство nn
topic_3: год который это исследование научный первый цена
topic_4: это россия страна который сша президент год
topic_5: форум писать связанный музыка бюро негативно ким
topic_6: это год который свой весь человек жизнь
topic_7: женщина золото женский профессия ми уральский мужчина
topic_8: земля гражданин температура запуск космос участок площадь
topic_9: газ это говорить всё мочь the газета
topic_10: год это который россия ребёнок система фонд
topic_11: двигатель это воздух день погода вирус дождь
topic_12: энергия журнал испытание влиять общество ск это
topic_13: ракета пенсия рейс лаборатория китай захватить сша
topic_14: погибнуть район армия результат земля авария километр
topic_15: российский украина станция россия экипаж граница год
topic_16: восток характерный полоса диапазон держаться фаза игнорировать
top

In [26]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [27]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.0,0.000000,0.120899,0.0,0.000000,0.067246,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.803771,0.000000,0.0
1,4896,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.257865,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.429058,0.0
2,4897,0.0,0.000000,0.0,0.273001,0.000000,0.0,0.000000,0.272461,0.000000,...,0.0,0.0,0.000000,0.0,0.116974,0.0,0.0,0.000000,0.000000,0.0
3,4898,0.0,0.029171,0.0,0.000000,0.000000,0.0,0.217399,0.100999,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.057794,0.000000,0.0
4,4899,0.0,0.000000,0.0,0.000000,0.372170,0.0,0.000000,0.098963,0.108483,...,0.0,0.0,0.163787,0.0,0.000000,0.0,0.0,0.082946,0.000000,0.0


In [29]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))
doc_dict[293622]

array([0.08438673, 0.        , 0.10556173, 0.13255735, 0.19691202,
       0.        , 0.21326303, 0.        , 0.        , 0.06475788,
       0.        , 0.        , 0.        , 0.02076281, 0.        ,
       0.06408942, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.10798933])

## Модифицировать код функции get_user_embedding

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

In [32]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, metric=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    return user_vector

In [34]:
get_user_embedding(user_articles_list, np.median)

array([0.        , 0.        , 0.05921824, 0.        , 0.2773228 ,
       0.        , 0.05407586, 0.        , 0.        , 0.        ,
       0.05580631, 0.        , 0.        , 0.        , 0.        ,
       0.01392017, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01072808])

In [69]:
# Функция расчета метрик
def get_metrics(metric):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]
    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                        X['churn'], shuffle=True, train_size=0.7, random_state=42)

    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)
    ix = np.argmax(fscore)
    
    return roc_auc, precision[ix], recall[ix], fscore[ix]

**mean**

In [70]:
m = list(get_metrics(np.mean))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.923, precision = 0.525, recall = 0.765, fscore = 0.623


**median**

In [71]:
m = list(get_metrics(np.median))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.943, precision = 0.611, recall = 0.746, fscore = 0.672


## Метрика max

Повторить п.2, но используя уже не медиану, а max

**max**

In [72]:
m = list(get_metrics(np.max))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.975, precision = 0.738, recall = 0.870, fscore = 0.798


## Таблица метрик

Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [61]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
info = pd.DataFrame(columns=metrics.keys())

In [73]:
for metric, value in metrics.items():
    print(metric, value)
    roc_auc, precision, recall, fscore = get_metrics(value)
    info.loc[f'roc_auc', metric] = roc_auc
    info.loc[f'precision', metric] = precision
    info.loc[f'recall', metric] = recall
    info.loc[f'fscore', metric] = fscore
    
info

mean <function mean at 0x00000174FD3A9B88>
median <function median at 0x00000174FD506828>
max <function amax at 0x00000174FD397948>


,mean,median,max
roc_auc,0.922987,0.943186,0.974805
precision,0.524554,0.610667,0.737569
recall,0.765472,0.745928,0.869707
fscore,0.622517,0.671554,0.798206
